In [ ]:
def register_file_upload_callback(app, name):
    @app.callback(
        Output('uploaded-file-' + name, 'data'),  # Changed to match your store ID
        Input('upload-data-' + name, 'contents'),
        State('upload-data-' + name, 'filename'),
        prevent_initial_call=True  # Add this to prevent initial trigger
    )
    def upload_files(contents, filenames):
        print(f"\n=== File Upload Callback Triggered for {name} ===")
        print(f"Contents received: {True if contents else False}")
        print(f"Filenames: {filenames}")

        if contents is None:
            raise dash.exceptions.PreventUpdate

        # Convert single file to list format
        if not isinstance(contents, list):
            contents = [contents]
            filenames = [filenames]

        encoded_files = []
        file_contents = []
        file_names = []

        for content, filename in zip(contents, filenames):
            if content is None:
                continue
                
            print(f"Processing {filename}")
            try:
                # Keep original content for PDFs
                if filename.lower().endswith('.pdf'):
                    encoded_files.append(content)
                    file_contents.append("")
                    file_names.append(filename)
                    print(f"Added PDF: {filename}")
                elif filename.lower().endswith('.txt'):
                    encoded_files.append(content)
                    try:
                        content_type, content_string = content.split(',')
                        decoded = base64.b64decode(content_string)
                        text = decoded.decode('utf-8')
                        file_contents.append(text)
                    except:
                        file_contents.append("")
                    file_names.append(filename)
                    print(f"Added text file: {filename}")

            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue

        print(f"Processed {len(encoded_files)} files")
        return encoded_files, file_contents, file_names

def register_update_tabs_callback(app, name):
    @app.callback(
        Output('file-tabs-' + name, 'tabs'),
        Output('file-tabs-' + name, 'children'),
        Input('uploaded-file-' + name, 'data'),
        Input('file-tabs-' + name, 'active_tab'),
        prevent_initial_call=True  # Add this to prevent initial trigger
    )
    def update_tabs(data, active_tab):
        print(f"\n=== Update Tabs Callback Triggered for {name} ===")
        print(f"Data received: {data is not None}")
        
        if not data:
            raise dash.exceptions.PreventUpdate

        encoded_files, file_contents, file_names = data
        print(f"Processing {len(encoded_files)} files: {file_names}")

        if not encoded_files:
            return [], []

        tabs = []
        children = []

        for i, (filename, encoded_file) in enumerate(zip(file_names, encoded_files)):
            tab_id = f'tab{i + 1}-{name}'
            tabs.append({
                'id': tab_id,
                'title': filename
            })
            
            children.append(
                html.Div(
                    html.Iframe(
                        id=f'pdf-viewer-{tab_id}',
                        src=encoded_file,
                        style={
                            'width': '100%',
                            'height': '800px',
                            'border': 'none',
                        }
                    ),
                    style={
                        'padding': '20px',
                        'height': '900px'
                    }
                )
            )
            print(f"Added tab for {filename}")

        return tabs, children

In [ ]:
def setup_layout(name):
    return html.Div(
        style={
            'width': '100%',
            'height': '100%',
            'margin': 0,
            'padding': 0,
            'overflow': 'hidden'
        },
        children=[
            uitk.HeaderAllIn(
                menu_items=[{
                    'type': 'outlined',
                    'id': 'unique-id' + name,
                }],
                brand={'appName': f'{name} BYOD', 'envBadge': {'name': f'{ENV}'}},
                search_expand_mode='none'
            ),
            html.Div(id='page-load-div' + name, style={'display': 'none'}),
            html.Div(
                children=[
                    uitk.Row(
                        children=[
                            uitk.Col(
                                children=[
                                    html.Div(
                                        children=[
                                            dcc.Upload(
                                                id='upload-data' + name,
                                                children=html.Div(['Drag and Drop or ', html.A('Select Files (.txt, .pdf)')]),
                                                style={
                                                    'width': '100%',
                                                    'height': '60px',
                                                    'lineHeight': '60px',
                                                    'borderWidth': '1px',
                                                    'borderStyle': 'dashed',
                                                    'borderRadius': '5px',
                                                    'textAlign': 'center',
                                                    'margin': '10px'
                                                },
                                                multiple=True
                                            ),
                                            uitk.DynamicTabs(
                                                id='file-tabs' + name,
                                                active_tab='tab1' + name,
                                                tabs=[{}]
                                            ),
                                        ]
                                    ),
                                ]
                            ),
                            uitk.Col(
                                children=[
                                    dbc.Container(
                                        children=[
                                            dcc.Store(id='store-conversation' + name, data=""),
                                            get_conversations_html(name),
                                            get_controls_html(name),
                                            dbc.Spinner(html.Div(id='loading-component' + name))
                                        ]
                                    )
                                ],
                                style={'width': 'auto'}
                            ),
                        ],
                        style={'padding': '10px'}
                    ),
                    html.Div(id='hidden-div' + name, style={'display': 'none'}),
                    uitk.Spinner(
                        html.Div(
                            id='storing-file' + name,
                            children=[dcc.Store(id='uploaded-file' + name)],
                            style={'minHeight': '50px'}
                        )
                    )
                ]
            )
        ]
    )